# Blockhash Oracle Configuration Script

This notebook configures the deployed blockhash oracle contracts.
Configuration state is parsed from RPC each time - no state files are maintained.

## 1. Configuration

In [ ]:
%load_ext autoreload
%autoreload 2
FORCE_PARIS = True
# Network type selection
NETWORK_TYPE = "mainnets"  # "testnets" or "mainnets"

# Configuration mode
# - "full": Configure all deployed chains and peers
# - "auto": Configure all deployed chains
# - "manual": Only configure chains listed in CHAINS_TO_CONFIGURE
CONFIGURATION_MODE = "full"

# For manual mode, specify which chains to configure
# Example: ["base-sepolia", "optimism-sepolia"]
READ_CHAINS = ["optimism", "polygon", "base", "arbitrum", "avalanche", "bsc"]
# READ_CHAINS = ["base-sepolia", "optimism-sepolia"]

CHAINS_TO_CONFIGURE = READ_CHAINS  # + ["etherlink"]

# Force reconfiguration options
FORCE_RECONFIGURE_PEERS = False  # Force re-set all peer relationships

SET_SEND_LIBS = True

## 2. Initialize Environment

In [ ]:
import json
import os
import sys
import logging
import subprocess
import time
from pathlib import Path

import boa
from dotenv import load_dotenv
from eth_account import Account
from web3 import Web3

# Add parent directory to path for imports
sys.path.append(str(Path().resolve().parent))
from ABIs import endpointV2_abi

# Import from deployment folder
from LZMetadata import LZMetadata
from DeploymentManager import DeploymentManager

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Load environment variables
load_dotenv()

# Constants
READ_CHANNEL_ID = 4294967295  # max uint32
CONFIG_TYPE_READ = 1
CONFIG_TYPE_ULN = 2
CONFIG_TYPE_EXECUTOR = 1

## 3. Helper Functions

In [ ]:
def get_vyper_abi(filepath):
    """Get ABI from Vyper contract file"""
    command = ["vyper", filepath, "-f", "abi_python"]
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr}"


def send_tx_single(w3, func, acc, value=0, gas=0):
    """Send transaction with retry logic"""

    try:
        tx = func.build_transaction(
            {
                "from": acc.address,
                "nonce": w3.eth.get_transaction_count(acc.address),
                "value": value,
            }
        )
    except Exception:
        tx = func.build_transaction(
            {
                "from": acc.address,
                "nonce": w3.eth.get_transaction_count(acc.address),
                "value": value,
                "gasPrice": int(1.1 * w3.eth.gas_price),
            }
        )
    if gas > 0:
        tx["gas"] = gas
    else:
        try:
            tx["gas"] = int(w3.eth.estimate_gas(tx) * 1.2)
        except Exception as e:
            if "no data" in str(e):
                tx["gas"] = 1_000_000
            else:
                raise (e)

    signed_tx = w3.eth.account.sign_transaction(tx, private_key=acc.key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    if receipt.status == 0:
        raise Exception("Transaction failed")
    return tx_hash


def send_tx(w3, func, acc, value=0, gas=0):
    """Send transaction with retry logic"""
    success = False
    fail_ctr = 0
    while not success and fail_ctr < 3:
        try:
            tx_hash = send_tx_single(w3, func, acc, value, gas)
            success = True
        except Exception as e:
            fail_ctr += 1
            if (
                "replacement transaction underpriced" in str(e)
                or "nonce too low" in str(e)
                or "could not replace existing tx" in str(e)
            ):
                print(str(e), "Retrying...")
                success = False
                time.sleep(1)
            else:
                raise e
    return tx_hash


def checksum(address):
    """Convert address to checksum format"""
    return Web3.to_checksum_address(address)

## 4. Load Deployment State

In [ ]:
# Load deployment state
deployment_manager = DeploymentManager()
deployed_contracts = deployment_manager.get_all_deployed_contracts(NETWORK_TYPE)

if not deployed_contracts:
    raise ValueError(f"No deployments found for {NETWORK_TYPE}")

logging.info(f"Found deployments for {len(deployed_contracts)} chains")

# Load chains configuration
with open("../chain-parse/chains.json", "r") as f:
    chains_config = json.load(f)

all_chains = chains_config[NETWORK_TYPE]

# Find main chain
main_chain = None
for chain_name, config in all_chains.items():
    if config.get("is_main_chain", False):
        main_chain = chain_name
        break

if not main_chain:
    raise ValueError(f"No main chain defined for {NETWORK_TYPE}")

logging.info(f"Main chain: {main_chain}")

# Determine which chains to configure
if CONFIGURATION_MODE == "full":
    # Configure all deployed chains
    chains_to_configure = set(deployed_contracts.keys())
elif CONFIGURATION_MODE == "manual":
    # Configure only specified chains
    chains_to_configure = set(CHAINS_TO_CONFIGURE)
    # Verify all specified chains have deployments
    for chain in chains_to_configure:
        if chain not in deployed_contracts:
            raise ValueError(f"Chain {chain} specified but no deployment found")
else:
    raise ValueError(f"Invalid CONFIGURATION_MODE: {CONFIGURATION_MODE}")

# Filter chains
chains = {k: v for k, v in all_chains.items() if k in chains_to_configure}

logging.info(f"Chains to configure: {list(chains.keys())}")

## 5. Setup Account

In [ ]:
# Get deployer account
if NETWORK_TYPE == "testnets":
    private_key = os.environ.get("WEB3_TESTNET_PK")
    if not private_key:
        raise ValueError("WEB3_TESTNET_PK not found in environment")
    account = Account.from_key(private_key)
else:
    # For mainnets, use secure key utilities
    sys.path.append(os.path.expanduser("~/projects/keys/scripts"))
    from secure_key_utils import get_web3_account
    from getpass import getpass

    ENCRYPTED_PK = os.environ.get("ENCRYPTED_PK")  # encrypted blob
    account = get_web3_account(ENCRYPTED_PK, getpass())  # decrypts with local pass + keychain

logging.info(f"Deployer address: {account.address}")

## 6. Initialize State with Deployed Contracts

In [ ]:
from web3.middleware import ExtraDataToPOAMiddleware

# Initialize LayerZero metadata
lz = LZMetadata()

# Initialize state dictionary
state_dict = {}

# Get ABIs
ABI_RELAY = get_vyper_abi("../../contracts/messengers/LZBlockRelay.vy")
ABI_ORACLE = get_vyper_abi("../../contracts/BlockOracle.vy")
ABI_HEADER_VERIFIER = get_vyper_abi("../../contracts/HeaderVerifier.vy")

# Load all deployed chains (including main chain if needed for read config)
all_deployed_chains = set(deployed_contracts.keys())
if main_chain in all_deployed_chains and main_chain not in chains:
    chains[main_chain] = all_chains[main_chain]

for chain_name, config in chains.items():
    logging.info(f"Configuring {chain_name}...")
    if chain_name not in deployed_contracts:
        logging.warning(f"No contracts deployed on {chain_name}, skipping")
        continue

    state_dict[chain_name] = {}

    # Setup RPC
    ankr_key = os.environ.get("ANKR_API_KEY")
    drpc_key = os.environ.get("DRPC_API_KEY")
    rpc_order = ["ankr", "drpc", "public"]
    for rpc_type in rpc_order:
        if rpc_type == "public" and NETWORK_TYPE == "testnets":
            rpc_type = "rpc"
        if config.get(rpc_type) is not None:
            if rpc_type == "ankr":
                rpc_url = config[rpc_type].format(ankr_key)
            elif rpc_type == "drpc":
                rpc_url = config[rpc_type].format(drpc_key)
            else:
                rpc_url = config[rpc_type]
            break
    else:
        raise ValueError(f"No RPC URL found for {chain_name}")

    # Setup boa environment
    boa.set_network_env(rpc_url)
    boa.env.add_account(account)

    # Store state
    state_dict[chain_name]["config"] = config
    state_dict[chain_name]["rpc"] = rpc_url
    state_dict[chain_name]["boa"] = boa.env
    state_dict[chain_name]["w3"] = Web3(Web3.HTTPProvider(rpc_url))
    state_dict[chain_name]["w3"].middleware_onion.inject(ExtraDataToPOAMiddleware, layer=0)

    state_dict[chain_name]["evm_version"] = "paris" if FORCE_PARIS else config["evm_version"]

    # Get LayerZero metadata
    try:
        lz_metadata = lz.get_chain_metadata(chain_name)
        state_dict[chain_name]["eid"] = lz_metadata["metadata"]["eid"]
        state_dict[chain_name]["endpoint"] = lz_metadata["metadata"]["endpointV2"]
        state_dict[chain_name]["send_lib"] = lz_metadata["metadata"].get(
            "sendUln302", "unavailable"
        )
        state_dict[chain_name]["receive_lib"] = lz_metadata["metadata"].get(
            "receiveUln302", "unavailable"
        )
        state_dict[chain_name]["read_lib"] = lz_metadata["metadata"].get(
            "readLib1002", "unavailable"
        )
        state_dict[chain_name]["dvns"] = lz_metadata["dvns"]
        state_dict[chain_name]["executor"] = lz_metadata["metadata"].get(
            "executor", "0x0000000000000000000000000000000000000000"
        )
    except Exception as e:
        logging.warning(f"Failed to get LZ metadata for {chain_name}: {e}")

    # Load deployed contracts
    contracts = deployed_contracts[chain_name]

    # Load contracts using boa
    with boa.swap_env(state_dict[chain_name]["boa"]):
        logging.info(f"Loading contracts for {chain_name}")
        if chain_name == main_chain:
            # Main chain has MainnetBlockView
            view_deployer = boa.load_partial(
                "../../contracts/MainnetBlockView.vy",
                compiler_args={"evm_version": state_dict[chain_name]["evm_version"]},
            )
            state_dict[chain_name]["oracle"] = view_deployer.at(contracts["MainnetBlockView"])
        else:
            # Other chains have BlockOracle, HeaderVerifier, and LZBlockRelay
            oracle_deployer = boa.load_partial(
                "../../contracts/BlockOracle.vy",
                compiler_args={"evm_version": state_dict[chain_name]["evm_version"]},
            )
            state_dict[chain_name]["oracle"] = oracle_deployer.at(contracts["BlockOracle"])

            verifier_deployer = boa.load_partial(
                "../../contracts/HeaderVerifier.vy",
                compiler_args={"evm_version": state_dict[chain_name]["evm_version"]},
            )
            state_dict[chain_name]["header_verifier"] = verifier_deployer.at(
                contracts["HeaderVerifier"]
            )

            relay_deployer = boa.load_partial(
                "../../contracts/messengers/LZBlockRelay.vy",
                compiler_args={"evm_version": state_dict[chain_name]["evm_version"]},
            )
            state_dict[chain_name]["block_relay"] = relay_deployer.at(contracts["LZBlockRelay"])

            # Also setup Web3 contracts for delegate operations
            state_dict[chain_name]["block_relay_w3"] = state_dict[chain_name]["w3"].eth.contract(
                address=contracts["LZBlockRelay"], abi=ABI_RELAY
            )
            state_dict[chain_name]["oracle_w3"] = state_dict[chain_name]["w3"].eth.contract(
                address=contracts["BlockOracle"], abi=ABI_ORACLE
            )
            state_dict[chain_name]["header_verifier_w3"] = state_dict[chain_name][
                "w3"
            ].eth.contract(address=contracts["HeaderVerifier"], abi=ABI_HEADER_VERIFIER)
            state_dict[chain_name]["endpoint_w3"] = state_dict[chain_name]["w3"].eth.contract(
                address=state_dict[chain_name]["endpoint"], abi=endpointV2_abi
            )

logging.info("Loaded all deployed contracts")

## 7. Configure Peers

In [ ]:
# find read supported chains
for chain_name in state_dict.keys():
    if state_dict[chain_name]["read_lib"] != "unavailable":
        print(chain_name)

In [ ]:
# filter out undesired read chains
for chain_name in state_dict.keys():
    if chain_name not in READ_CHAINS:
        if state_dict[chain_name]["read_lib"] != "unavailable":
            state_dict[chain_name]["read_lib"] = "unavailable"

In [ ]:
# Configure peers for all block relay contracts
# This checks existing configuration from RPC and only sets what's needed

for chain_name in chains_to_configure:
    if chain_name == main_chain:
        continue
    try:
        # Use dict to manage unique peer relationships
        desired_peers = {}

        # 1. If read-enabled: add all other chains as peers for broadcasting
        if state_dict[chain_name]["read_lib"] != "unavailable":
            for target_key in all_deployed_chains:
                if target_key != main_chain and target_key != chain_name:
                    if target_key in state_dict:
                        desired_peers[state_dict[target_key]["eid"]] = state_dict[target_key][
                            "block_relay"
                        ].address

        # 2. For all chains: add read-enabled chains as peers (to receive their broadcasts)
        for source_key in all_deployed_chains:
            if (
                source_key != main_chain
                and source_key != chain_name
                and source_key in state_dict
                and state_dict[source_key]["read_lib"] != "unavailable"
            ):
                desired_peers[state_dict[source_key]["eid"]] = state_dict[source_key][
                    "block_relay"
                ].address

        if not desired_peers:
            logging.info(f"No peers needed for {chain_name}")
            continue

        with boa.swap_env(state_dict[chain_name]["boa"]):
            relay_contract = state_dict[chain_name]["block_relay"]
            relay_contract_w3 = state_dict[chain_name]["block_relay_w3"]

            # Check which peers need to be set
            to_set_eids = []
            to_set_peers = []

            for peer_eid, peer_address in desired_peers.items():
                current_peer = relay_contract_w3.functions.peers(peer_eid).call()

                # Check if peer needs to be set or updated
                if (
                    FORCE_RECONFIGURE_PEERS
                    or peer_address[2:].lower() not in str(current_peer.hex()).lower()
                ):
                    to_set_eids.append(peer_eid)
                    to_set_peers.append(peer_address)

            # Set peers if needed
            if len(to_set_eids) > 0:
                logging.info(f"Setting {len(to_set_eids)} peers on {chain_name}...")
                relay_contract.set_peers(to_set_eids, to_set_peers)
                logging.info(f"Set {len(to_set_eids)} peers on {chain_name}")
            else:
                logging.info(f"All {len(desired_peers)} peers already configured on {chain_name}")
    except Exception as e:
        logging.error(f"Error configuring peers on {chain_name}: {e}")
        continue

## 8. Configure Relay-Oracle-HeaderVerifier

In [ ]:
# Configure block oracles and committers
for chain_name in chains_to_configure:
    try:
        if chain_name == main_chain:
            continue

        with boa.swap_env(state_dict[chain_name]["boa"]):
            relay_contract = state_dict[chain_name]["block_relay"]
            oracle_contract = state_dict[chain_name]["oracle"]
            header_verifier = state_dict[chain_name]["header_verifier"]
            relay_contract_w3 = state_dict[chain_name]["block_relay_w3"]
            oracle_contract_w3 = state_dict[chain_name]["oracle_w3"]
            header_verifier_w3 = state_dict[chain_name]["header_verifier_w3"]

            # Set block oracle on relay
            if relay_contract_w3.functions.block_oracle().call() != oracle_contract.address:
                relay_contract.set_block_oracle(oracle_contract.address)
                logging.info(f"Set block oracle on {chain_name}")
            else:
                logging.info(f"Block oracle already set on {chain_name}")

            # Add relay as committer to oracle
            if not oracle_contract_w3.functions.is_committer(relay_contract.address).call():
                oracle_contract.add_committer(relay_contract.address, True)
                logging.info(f"Added relay as committer on {chain_name}")
            else:
                logging.info(f"Relay already a committer on {chain_name}")

            # Set header verifier on oracle
            if oracle_contract_w3.functions.header_verifier().call() != header_verifier.address:
                oracle_contract.set_header_verifier(header_verifier.address)
                logging.info(f"Set header verifier on {chain_name}")
            else:
                logging.info(f"Header verifier already set on {chain_name}")
    except Exception as e:
        logging.error(f"Error configuring {chain_name}: {e}")
        continue

## 9. Configure Read Settings

In [ ]:
# Configure read settings for read-enabled chains
for chain_name in chains_to_configure:
    if chain_name == main_chain:
        continue

    if state_dict[chain_name]["read_lib"] != "unavailable":
        with boa.swap_env(state_dict[chain_name]["boa"]):
            relay_contract = state_dict[chain_name]["block_relay"]

            # Check current read config
            is_enabled = relay_contract.read_enabled()
            read_channel = relay_contract.read_channel()
            mainnet_eid = relay_contract.mainnet_eid()
            mainnet_view = relay_contract.mainnet_block_view()

            expected_mainnet_view = state_dict[main_chain]["oracle"].address
            expected_mainnet_eid = state_dict[main_chain]["eid"]

            if (
                not is_enabled
                or read_channel != READ_CHANNEL_ID
                or mainnet_eid != expected_mainnet_eid
                or mainnet_view != expected_mainnet_view
            ):
                logging.info(f"Setting read config on {chain_name}...")
                relay_contract.set_read_config(
                    True,
                    READ_CHANNEL_ID,
                    expected_mainnet_eid,
                    expected_mainnet_view,
                )
                logging.info(f"Read config set on {chain_name}")
            else:
                logging.info(f"Read config already set correctly on {chain_name}")

## 10. Configure LayerZero Libraries

In [ ]:
# Configure LayerZero send/receive libraries via endpoint delegate
FORCE_RECONFIGURE_LIBS = True
FORCE_RECONFIGURE_FOR_PEERS = []
all_eids = [state_dict[key]["eid"] for key in state_dict.keys()]


def chksum(address):
    return w3.to_checksum_address(address)


eid_to_name = {state_dict[key]["eid"]: key for key in state_dict.keys()}

for chain_name in chains_to_configure:
    if chain_name == main_chain:
        continue

    # Get configured peers
    relay_w3 = state_dict[chain_name]["block_relay_w3"]
    endpoint_w3 = state_dict[chain_name]["endpoint_w3"]
    w3 = state_dict[chain_name]["w3"]
    read_supported = state_dict[chain_name]["read_lib"] != "unavailable"

    eid_peers = {}
    for eid in all_eids:
        peer = relay_w3.functions.peers(eid).call().hex()
        if peer != "0000000000000000000000000000000000000000000000000000000000000000":
            eid_peers[eid] = "0x" + peer[24:]

    logging.info(f"\nConfiguring libraries on {chain_name}...")
    logging.info(f"Peers: {len(eid_peers)}")

    send_lib = state_dict[chain_name]["send_lib"]
    receive_lib = state_dict[chain_name]["receive_lib"]
    read_lib = state_dict[chain_name]["read_lib"]
    oapp = relay_w3.address

    # Configure send/receive libs for each peer
    for eid, _ in eid_peers.items():
        eid_name = eid_to_name.get(eid)
        peer_read_supported = state_dict[eid_name]["read_lib"] != "unavailable"
        # Check and set send library
        if SET_SEND_LIBS:
            if read_supported:
                try:
                    current_send_lib = endpoint_w3.functions.getSendLibrary(oapp, eid).call()
                    if (
                        FORCE_RECONFIGURE_LIBS
                        or chksum(current_send_lib) != chksum(send_lib)
                        or eid_name in FORCE_RECONFIGURE_FOR_PEERS
                    ):
                        call_fn = endpoint_w3.functions.setSendLibrary(oapp, eid, chksum(send_lib))
                        tx_hash = send_tx(w3, call_fn, account)
                        logging.info(f"Set send library for EID {eid}: {tx_hash.hex()}")
                except Exception as e:
                    logging.error(f"Error setting send library on {chain_name} for EID {eid}: {e}")
            else:
                logging.info(
                    f"Skipping {chain_name} -> {eid_name} - read not supported on {chain_name}"
                )

            # Check and set receive library
            if peer_read_supported:
                try:
                    current_receive_lib = endpoint_w3.functions.getReceiveLibrary(oapp, eid).call()[
                        0
                    ]
                    if (
                        peer_read_supported
                        and (FORCE_RECONFIGURE_LIBS or chain_name in FORCE_RECONFIGURE_FOR_PEERS)
                        or chksum(current_receive_lib) != chksum(receive_lib)
                    ):
                        call_fn = endpoint_w3.functions.setReceiveLibrary(
                            oapp, eid, chksum(receive_lib), 0
                        )
                        tx_hash = send_tx(w3, call_fn, account)
                        logging.info(f"Set receive library for EID {eid}: {tx_hash.hex()}")
                except Exception as e:
                    logging.error(
                        f"Error setting receive library on {chain_name} for EID {eid}: {e}"
                    )
            else:
                logging.info(
                    f"Skipping {eid_name} -> {chain_name} - read not supported on {eid_name}"
                )

    # Configure read library if available
    if read_lib != "unavailable":
        try:
            read_channel = relay_w3.functions.read_channel().call()
            if read_channel != 0:
                # Check send library for read channel
                current_send_lib = endpoint_w3.functions.getSendLibrary(oapp, read_channel).call()
                if FORCE_RECONFIGURE_LIBS or chksum(current_send_lib) != chksum(read_lib):
                    call_fn = endpoint_w3.functions.setSendLibrary(
                        oapp, read_channel, chksum(read_lib)
                    )
                    tx_hash = send_tx(w3, call_fn, account)
                    logging.info(f"Set send-read library: {tx_hash.hex()}")
                else:
                    logging.info(f"Send-read library already set on {chain_name}")

                # Check receive library for read channel
                current_receive_lib = endpoint_w3.functions.getReceiveLibrary(
                    oapp, read_channel
                ).call()[0]
                if chksum(current_receive_lib) != chksum(read_lib):
                    call_fn = endpoint_w3.functions.setReceiveLibrary(
                        oapp, read_channel, chksum(read_lib), 0
                    )
                    tx_hash = send_tx(w3, call_fn, account)
                    logging.info(f"Set receive-read library: {tx_hash.hex()}")
                else:
                    logging.info(f"Receive-read library already set on {chain_name}")
            else:
                logging.info(f"Read channel not set on {chain_name}")
        except Exception as e:
            logging.error(f"Error setting read library on {chain_name}: {e}")

## 13. Configuration Summary

In [ ]:
# Print configuration summary
print("\n" + "=" * 80)
print("CONFIGURATION SUMMARY")
print("=" * 80)
print(f"Network Type: {NETWORK_TYPE}")
print(f"Configuration Mode: {CONFIGURATION_MODE}")
print(f"Configured chains: {len(chains_to_configure)}")

# Show all deployed chains
print(f"\nAll deployed chains ({len(deployed_contracts)}):")
for chain_name in sorted(deployed_contracts.keys()):
    configured_marker = " [CONFIGURED]" if chain_name in chains_to_configure else ""
    print(f"  - {chain_name}{configured_marker}")

print("\nConfiguration details:")
for chain_name in sorted(chains_to_configure):
    if chain_name == main_chain:
        print(f"\n{chain_name} (Main Chain):")
        print(f"  MainnetBlockView: {state_dict[chain_name]['oracle'].address}")
    else:
        print(f"\n{chain_name}:")
        print(f"  BlockOracle: {state_dict[chain_name]['oracle'].address}")
        print(f"  HeaderVerifier: {state_dict[chain_name]['header_verifier'].address}")
        print(f"  LZBlockRelay: {state_dict[chain_name]['block_relay'].address}")

        # Check peer count
        with boa.swap_env(state_dict[chain_name]["boa"]):
            # configured_eids = state_dict[chain_name]["block_relay"].get_configured_eids()
            # configured_eids = get_configured_eids(state_dict[chain_name]["w3"], state_dict[chain_name]["block_relay_w3"])
            # print(f"  Configured peers: {len(configured_eids)}")

            # Check if read enabled
            if state_dict[chain_name]["read_lib"] != "unavailable":
                read_enabled = state_dict[chain_name]["block_relay"].read_enabled()
                if read_enabled:  # is_enabled
                    print(
                        f"  Read enabled: Yes (channel {state_dict[chain_name]["block_relay"].read_channel()})"
                    )

In [ ]:
def get_configured_eids(w3, contract):
    """
    Parses PeerSet logs to find all configured peer EIDs.
    A peer is considered configured if it's set to a non-zero address.
    """
    event_filter = contract.events.PeerSet.create_filter(from_block="0x0")
    logs = event_filter.get_all_entries()
    print(logs)


get_configured_eids(state_dict["base"]["w3"], state_dict["base"]["block_relay_w3"])